In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(31):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                type_ = html.xpath('//span[@id="lblPartTypeName"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                application_summary = ''.join(html.xpath('//div[@id="divAppSummary"]/p/text()')).strip().replace(', ', '\n')
                
                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//a[@id="ProductPic"]/@href')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                vehicle_url = html.xpath('//input[@id="hidfDetailApp"]/@value')
                if len(vehicle_url) == 0:
                    vehicle_url = ''
                else:
                    vehicle_url = 'https://www.dormanproducts.com/' + vehicle_url[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                oe = html.xpath('//section[@id="productOE"]/descendant::tbody/tr/th/text()')
                oe = ';'.join([_.strip() for _ in oe])
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//section[@id="productSpec"]/descendant::tr')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Part_Number': part_number,
                                         'Type': type_,
                                         'Description': description,
                                         'Application_Summary': application_summary,
                                         'Vehicle': '',
                                         'OE': oe,
                                         'Pic': '',
                                         'Url': url,
                                         'Vehicle_Url': vehicle_url,
                                         'Src': src}])
                
                for tr in list_tr:
                    df_temp.loc[0, tr.xpath('./th/text()')[0].strip()[:-1].lower()] = tr.xpath('./td/text()')[0].strip()

                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：1133

https://www.dormanproducts.com/p-82905-264-854.aspx?origin=keyword  <->  [ok] - 剩余数量：1113
https://www.dormanproducts.com/p-133906-264-568.aspx?origin=keyword  <->  [ok] - 剩余数量：1112
https://www.dormanproducts.com/p-133856-264-295.aspx?origin=keyword  <->  [ok] - 剩余数量：1111
https://www.dormanproducts.com/p-133780-264-156.aspx?origin=keyword  <->  [ok] - 剩余数量：1110
https://www.dormanproducts.com/p-133798-264-180.aspx?origin=keyword  <->  [ok] - 剩余数量：1109
https://www.dormanproducts.com/p-133774-264-145.aspx?origin=keyword  <->  [ok] - 剩余数量：1108
https://www.dormanproducts.com/p-80162-264-380.aspx?origin=keyword  <->  [ok] - 剩余数量：1107
https://www.dormanproducts.com/p-133965-264-188.aspx?origin=keyword  <->  [ok] - 剩余数量：1106
https://www.dormanproducts.com/p-133779-264-155.aspx?origin=keyword  <->  [ok] - 剩余数量：1105
https://www.dormanproducts.com/p-49839-264-724.aspx?origin=keyword  <->  [ok] - 剩余数量：1104
https://www.dormanproducts.com/p-72801-264-373.aspx?origin=keyword  <->  [ok] - 剩余数

https://www.dormanproducts.com/p-77641-264-458.aspx?origin=keyword  <->  [ok] - 剩余数量：1022
https://www.dormanproducts.com/p-74037-264-447.aspx?origin=keyword  <->  [ok] - 剩余数量：1021
https://www.dormanproducts.com/p-72776-264-445.aspx?origin=keyword  <->  [ok] - 剩余数量：1020
https://www.dormanproducts.com/p-65056-264-417.aspx?origin=keyword  <->  [ok] - 剩余数量：1019
https://www.dormanproducts.com/p-77791-264-346.aspx?origin=keyword  <->  [ok] - 剩余数量：1018
https://www.dormanproducts.com/p-73442-264-428.aspx?origin=keyword  <->  [ok] - 剩余数量：1017
https://www.dormanproducts.com/p-73453-264-434.aspx?origin=keyword  <->  [ok] - 剩余数量：1016
https://www.dormanproducts.com/p-77888-264-464.aspx?origin=keyword  <->  [ok] - 剩余数量：1015
https://www.dormanproducts.com/p-73357-264-429.aspx?origin=keyword  <->  [ok] - 剩余数量：1014
https://www.dormanproducts.com/p-73267-264-359.aspx?origin=keyword  <->  [ok] - 剩余数量：1013
https://www.dormanproducts.com/p-82765-264-486.aspx?origin=keyword  <->  [ok] - 剩余数量：1012
https://ww

https://www.dormanproducts.com/p-133764-264-099.aspx?origin=keyword  <->  [ok] - 剩余数量：929
https://www.dormanproducts.com/p-135569-264-885.aspx?origin=keyword  <->  [ok] - 剩余数量：928
https://www.dormanproducts.com/p-133899-264-487.aspx?origin=keyword  <->  [ok] - 剩余数量：927
https://www.dormanproducts.com/p-133833-264-264.aspx?origin=keyword  <->  [ok] - 剩余数量：926
https://www.dormanproducts.com/p-52208-65399.aspx?origin=keyword  <->  [ok] - 剩余数量：925
https://www.dormanproducts.com/p-102392-090-183cd.aspx?origin=keyword  <->  [ok] - 剩余数量：924
https://www.dormanproducts.com/p-30140-65310.aspx?origin=keyword  <->  [ok] - 剩余数量：923
https://www.dormanproducts.com/p-5590-65292.aspx?origin=keyword  <->  [ok] - 剩余数量：922
https://www.dormanproducts.com/p-20353-65269.aspx?origin=keyword  <->  [ok] - 剩余数量：921
https://www.dormanproducts.com/p-28672-65206.aspx?origin=keyword  <->  [ok] - 剩余数量：920
https://www.dormanproducts.com/p-20349-65212.aspx?origin=keyword  <->  [ok] - 剩余数量：919
https://www.dormanproducts.

https://www.dormanproducts.com/p-103921-092-018.aspx?origin=keyword  <->  [ok] - 剩余数量：835
https://www.dormanproducts.com/p-19504-097-129.aspx?origin=keyword  <->  [ok] - 剩余数量：834
https://www.dormanproducts.com/p-1440-097-027.aspx?origin=keyword  <->  [ok] - 剩余数量：833
https://www.dormanproducts.com/p-133-090-0821.aspx?origin=keyword  <->  [ok] - 剩余数量：832
https://www.dormanproducts.com/p-26547-090-0351.aspx?origin=keyword  <->  [ok] - 剩余数量：831
https://www.dormanproducts.com/p-25756-097-0271.aspx?origin=keyword  <->  [ok] - 剩余数量：830
https://www.dormanproducts.com/p-1136-095-1431.aspx?origin=keyword  <->  [ok] - 剩余数量：829
https://www.dormanproducts.com/p-26491-090-0311.aspx?origin=keyword  <->  [ok] - 剩余数量：828
https://www.dormanproducts.com/p-26832-65282.aspx?origin=keyword  <->  [ok] - 剩余数量：827
https://www.dormanproducts.com/p-49975-264-321.aspx?origin=keyword  <->  [ok] - 剩余数量：826
https://www.dormanproducts.com/p-66793-264-244.aspx?origin=keyword  <->  [ok] - 剩余数量：825
https://www.dormanpro

https://www.dormanproducts.com/p-17618-264-005.aspx?origin=keyword  <->  [ok] - 剩余数量：743
https://www.dormanproducts.com/p-1849-264-038.aspx?origin=keyword  <->  [ok] - 剩余数量：742
https://www.dormanproducts.com/p-53561-264-233.aspx?origin=keyword  <->  [ok] - 剩余数量：741
https://www.dormanproducts.com/p-47702-264-238.aspx?origin=keyword  <->  [ok] - 剩余数量：740
https://www.dormanproducts.com/p-65044-264-080.aspx?origin=keyword  <->  [ok] - 剩余数量：739
https://www.dormanproducts.com/p-65325-264-062.aspx?origin=keyword  <->  [ok] - 剩余数量：738
https://www.dormanproducts.com/p-49845-264-411.aspx?origin=keyword  <->  [ok] - 剩余数量：737
https://www.dormanproducts.com/p-73381-264-433.aspx?origin=keyword  <->  [ok] - 剩余数量：736
https://www.dormanproducts.com/p-2851-264-210.aspx?origin=keyword  <->  [ok] - 剩余数量：735
https://www.dormanproducts.com/p-46823-264-132.aspx?origin=keyword  <->  [ok] - 剩余数量：734
https://www.dormanproducts.com/p-82082-264-374.aspx?origin=keyword  <->  [ok] - 剩余数量：733
https://www.dormanprodu

...

https://www.dormanproducts.com/p-103007-090-841cd.aspx?origin=keyword  <->  [ok] - 剩余数量：372
https://www.dormanproducts.com/p-73449-69013.aspx?origin=keyword  <->  [ok] - 剩余数量：371
https://www.dormanproducts.com/p-74014-69016.aspx?origin=keyword  <->  [ok] - 剩余数量：370
https://www.dormanproducts.com/p-1125-090-0341.aspx?origin=keyword  <->  [ok] - 剩余数量：369
https://www.dormanproducts.com/p-132-090-0791.aspx?origin=keyword  <->  [ok] - 剩余数量：368
https://www.dormanproducts.com/p-73510-69012.aspx?origin=keyword  <->  [ok] - 剩余数量：367
https://www.dormanproducts.com/p-101349-097-116cd.aspx?origin=keyword  <->  [ok] - 剩余数量：366
https://www.dormanproducts.com/p-1775-090-1531.aspx?origin=keyword  <->  [ok] - 剩余数量：365
https://www.dormanproducts.com/p-695-097-139.aspx?origin=keyword  <->  [ok] - 剩余数量：364
https://www.dormanproducts.com/p-359-090-0881.aspx?origin=keyword  <->  [ok] - 剩余数量：363
https://www.dormanproducts.com/p-102243-095-142cd.aspx?origin=keyword  <->  [ok] - 剩余数量：362
https://www.dormanprod

https://www.dormanproducts.com/p-104982-090-5007cd.aspx?origin=keyword  <->  [ok] - 剩余数量：279
https://www.dormanproducts.com/p-101376-097-022cd.aspx?origin=keyword  <->  [ok] - 剩余数量：278
https://www.dormanproducts.com/p-26507-097-119.aspx?origin=keyword  <->  [ok] - 剩余数量：277
https://www.dormanproducts.com/p-49465-264-716.aspx?origin=keyword  <->  [ok] - 剩余数量：276
https://www.dormanproducts.com/p-871-090-070.aspx?origin=keyword  <->  [ok] - 剩余数量：275
https://www.dormanproducts.com/p-102382-090-933cd.aspx?origin=keyword  <->  [ok] - 剩余数量：274
https://www.dormanproducts.com/p-106029-090-5010cd.aspx?origin=keyword  <->  [ok] - 剩余数量：273
https://www.dormanproducts.com/p-1145-097-135.aspx?origin=keyword  <->  [ok] - 剩余数量：272
https://www.dormanproducts.com/p-52196-65396.aspx?origin=keyword  <->  [ok] - 剩余数量：271
https://www.dormanproducts.com/p-133964-264-098.aspx?origin=keyword  <->  [ok] - 剩余数量：270
https://www.dormanproducts.com/p-12143-65271.aspx?origin=keyword  <->  [ok] - 剩余数量：269
https://www.d

https://www.dormanproducts.com/p-106067-264-575.aspx?origin=keyword  <->  [ok] - 剩余数量：185
https://www.dormanproducts.com/p-80561-264-468.aspx?origin=keyword  <->  [ok] - 剩余数量：184
https://www.dormanproducts.com/p-107488-264-632.aspx?origin=keyword  <->  [ok] - 剩余数量：183
https://www.dormanproducts.com/p-106824-264-538.aspx?origin=keyword  <->  [ok] - 剩余数量：182
https://www.dormanproducts.com/p-106239-264-633.aspx?origin=keyword  <->  [ok] - 剩余数量：181
https://www.dormanproducts.com/p-136307-264-865.aspx?origin=keyword  <->  [ok] - 剩余数量：180
https://www.dormanproducts.com/p-133813-264-211.aspx?origin=keyword  <->  [ok] - 剩余数量：179
https://www.dormanproducts.com/p-133763-264-097.aspx?origin=keyword  <->  [ok] - 剩余数量：178
https://www.dormanproducts.com/p-77883-264-381.aspx?origin=keyword  <->  [ok] - 剩余数量：177
https://www.dormanproducts.com/p-133799-264-181.aspx?origin=keyword  <->  [ok] - 剩余数量：176
https://www.dormanproducts.com/p-133787-264-169.aspx?origin=keyword  <->  [ok] - 剩余数量：175
https://www.

https://www.dormanproducts.com/p-64692-264-243.aspx?origin=keyword  <->  [ok] - 剩余数量：92
https://www.dormanproducts.com/p-2026-264-606.aspx?origin=keyword  <->  [ok] - 剩余数量：91
https://www.dormanproducts.com/p-65055-264-077.aspx?origin=keyword  <->  [ok] - 剩余数量：90
https://www.dormanproducts.com/p-86199-264-531.aspx?origin=keyword  <->  [ok] - 剩余数量：89
https://www.dormanproducts.com/p-135572-264-888.aspx?origin=keyword  <->  [ok] - 剩余数量：88
https://www.dormanproducts.com/p-53361-264-242.aspx?origin=keyword  <->  [ok] - 剩余数量：87
https://www.dormanproducts.com/p-50058-264-507.aspx?origin=keyword  <->  [ok] - 剩余数量：86
https://www.dormanproducts.com/p-64607-264-161.aspx?origin=keyword  <->  [ok] - 剩余数量：85
https://www.dormanproducts.com/p-106074-264-560.aspx?origin=keyword  <->  [ok] - 剩余数量：84
https://www.dormanproducts.com/p-106255-264-589.aspx?origin=keyword  <->  [ok] - 剩余数量：83
https://www.dormanproducts.com/p-65334-264-512.aspx?origin=keyword  <->  [ok] - 剩余数量：82
https://www.dormanproducts.com

https://www.dormanproducts.com/p-7165-45680.aspx?origin=keyword  <->  [ok] - 剩余数量：0
https://www.dormanproducts.com/p-64611-264-325.aspx?origin=keyword  <->  [ok] - 剩余数量：0
https://www.dormanproducts.com/p-52322-66003.aspx?origin=keyword  <->  [ok] - 剩余数量：0
https://www.dormanproducts.com/p-2717-097-017.aspx?origin=keyword  <->  [ok] - 剩余数量：0
https://www.dormanproducts.com/p-6194-963-058.aspx?origin=keyword  <->  [ok] - 剩余数量：0
https://www.dormanproducts.com/p-102267-095-140cd.aspx?origin=keyword  <->  [ok] - 剩余数量：0
https://www.dormanproducts.com/p-134668-264-061.aspx?origin=keyword  <->  [ok] - 剩余数量：0
https://www.dormanproducts.com/p-100954-963-232d.aspx?origin=keyword  <->  [ok] - 剩余数量：0
https://www.dormanproducts.com/p-376-097-012.aspx?origin=keyword  <->  [ok] - 剩余数量：0
https://www.dormanproducts.com/p-3739-65306.aspx?origin=keyword  <->  [ok] - 剩余数量：0
https://www.dormanproducts.com/p-22446-65245.aspx?origin=keyword  <->  [ok] - 剩余数量：0
https://www.dormanproducts.com/p-72805-264-334.aspx